# Cassandra - LJ example

environment setup

* switched to conda mosdef cassandra

`conda create -n cassandra -y -c conda-forge -c mosdef -c omnia 'python=3.7' mbuild foyer openbabel fresnel py3dmol 'nodejs>=10' jupyterlab mosdef_cassandra matplotlib` 

pip installed gmso

`conda activate cassandra`

In [1]:
import mosdef_cassandra as mc
#import gmso
import foyer
import mbuild as mb
import mosdef_cassandra.examples as ex
import numpy as np

If the cell below fails, you need to add Cassandra to your path

```
export PATH=/Users/$(whoami)/Projects/Cassandra_V1.2/bin:${PATH}
```

In [2]:
mc.utils.detect_cassandra_binaries();

Using the following executables for Cassandra:
Python: /Users/jenny/miniconda3/envs/cassandra/bin/python
library_setup: /Users/jenny/miniconda3/envs/cassandra/bin/library_setup.py
Cassandra: /Users/jenny/miniconda3/envs/cassandra/bin/cassandra.exe


In [3]:
lj_ff = foyer.Forcefield()
lj_ff.loadFile("FF_xmls/LJ.xml")

/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/foyer/forcefield.py:457: UserWarning: Non-atomistic element type detected. Creating custom element for 
  'Creating custom element for {}'.format(element))


In [6]:
ljium = mb.Compound(name="C", pos=[0,0,0])
#ljium.visualize().show()

In [14]:
typed_ljium = lj_ff.apply(
    ljium, assert_bond_params=False, assert_angle_params=False, 
    assert_dihedral_params=False
)
L = 10
box = mb.Box(mins=[-L/2]*3, maxs=[L/2]*3)

/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/parmed/openmm/topsystem.py:238: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  'all be present in one form or another.', OpenMMWarning)


In [20]:
%%bash
# Delete output of previous run
rm -rf *.in.* species* *.out* *.inp* *.log

In [21]:
# Create box and species list
box_list = [box]
species_list = [typed_ljium]

# Since we have an occupied box we need to specify
# the number of each species present in the intial config
mols_in_boxes = [[0]]

system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
moves = mc.Moves("gcmc", species_list)

custom_args = {
    "chemical_potentials": [-4],
    "rcut_min": 0.5,
    "vdw_cutoff": min(box.lengths)/2.1,
    "charge_style": "none",
    #"charge_cutoff": 14.0,
    "coord_freq": 100,
    "prop_freq": 10,
}

mc.run(system, moves, "equilibration", 10000, 300.0, **custom_args)

Using the following executables for Cassandra:
Python: /Users/jenny/miniconda3/envs/cassandra/bin/python
library_setup: /Users/jenny/miniconda3/envs/cassandra/bin/library_setup.py
Cassandra: /Users/jenny/miniconda3/envs/cassandra/bin/cassandra.exe
Generating fragment libraries...
Running Cassandra...


/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/formats/cassandramcf.py:178: UserWarning: No bonds found. Cassandra will interpet this as a rigid species
  warnings.warn("No bonds found. Cassandra will interpet "


In [25]:
%%bash

#mkdir lj_results
#mv -f *.in.* species* *.out* *.inp* *.log lj_results

In [26]:
import ex_render_cassandra as erc

In [39]:
import freud 
from ex_render_cassandra import Cassandra_frame

def make_traj(xyzfile, boxfile):
    d_xyz = np.dtype([("atom", np.unicode_, 8), ("xyz", "d", 3)])

    with open(xyzfile, "r") as f:
        xyzlines = f.readlines()

    with open(boxfile, "r") as f:
        boxlines = f.readlines()

    i = 0
    boxes = []
    while True:
        try:
            line = boxlines[i]
        except IndexError:
            break
        if len(line.split()) == 3:
            boxmat = np.array(
                [line.split(),boxlines[i+1].split(), boxlines[i+2].split()],
                dtype="float64"
            )
            boxes.append(freud.box.Box.from_matrix(boxmat))
            i += 3
        i += 1

    traj = []
    i = 0
    step = 0
    while True:
        try:
            n_atoms = int(xyzlines[i])
        except IndexError:
            break
        #print(xyzlines[i+1]) # this should print the step
        arr = xyzlines[i+2:i+2+n_atoms]
        data = np.genfromtxt(arr, dtype = d_xyz)
        atoms = np.array([i["atom"] for i in data], dtype="U8")
        try:
            xyz = np.stack([i["xyz"] for i in data])
            frame = Cassandra_frame(atoms, xyz, boxes[step])
            traj.append(frame)
        except ValueError:
            pass
        i += 2+n_atoms
        step += 1
    return traj

In [40]:
xyzfile = "lj_results/gcmc.out.xyz"
boxfile = "lj_results/gcmc.out.H"
traj = erc.make_traj(xyzfile, boxfile)

/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: genfromtxt: Empty input file: "[]"


In [41]:
# slow!
erc.display_movie(erc.render_sphere_frame, traj, gif="lj_results/traj.gif")

![](lj_results/traj.gif)